In [1]:
import sys
sys.path.append('/Users/busalels/Desktop/KAUST/spring2024/research')

In [2]:
from scipy import integrate
import sympy as sp
import numpy as np
from numpy import linalg as la 
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML
font = {'size'   : 15}
matplotlib.rc('font', **font)
from New_Homogenized_system_coefficients1_pwc import C_values, Homogenized_system_coef
fft = np.fft.fft
ifft = np.fft.ifft
from tqdm import tqdm 

In [3]:
def rk3(v,xi,rhs,dt):
    y2 = v + dt*rhs(v,xi)
    y3 = 0.75*v + 0.25*(y2 + dt*rhs(y2,xi))
    v_new = 1./3 * v + 2./3 * (y3 + dt*rhs(y3,xi))
    return v_new

In [4]:
num_plots =  25

In [5]:
def solve_isentropic_gas_equations(m, dt, num_plots):
    L = 100  
    x = np.arange(-m/2, m/2) * (L/m)
    xi = np.fft.fftfreq(m) * m * 2 * np.pi / L

    if cross_section == 'piecewise':
        Amp = 1/20
        rho0 = 0.3 + Amp*np.exp(-1*(x/8)**2)
    elif cross_section == 'variable':
        Amp = 1/12 
        rho0 = p_0 + Amp * np.exp(-(x/5)**2) 
    elif cross_section == 'pwc_small':
        Amp = 1/40
        rho0 = 0.3 + Amp*np.exp(-1*(x/10)**2)
    q0 = np.zeros_like(rho0)
    drhodx = np.zeros_like(rho0)
    dqdx = np.zeros_like(rho0)
    drhodxt = np.real(ifft(1j * xi * fft(rho0)))
    dqdxt = np.real(ifft(1j * xi * fft(q0)))
    v = np.zeros((2, len(x)))
    v[0, :] = rho0; v[1, :] = q0
    vhat = np.zeros_like(v)
    vhat[0, :] = drhodxt; vhat[1, :] = dqdxt
    tmax = 50
    
    def rhs(v, xi):
        d = np.zeros_like(v)
        rho = v[0, :]
        q = v[1, :]
        q_hat = fft(q)
        rho_hat = fft(rho)
        drhodx = np.real(ifft(1j * xi * rho_hat))
        drho2dx2 = np.real(ifft((1j * xi)**2 * rho_hat ))
        drho3dx3 = np.real(ifft((1j * xi)**3 * rho_hat ))
        dqdx  = np.real(ifft(1j * xi * q_hat))
        dq2dx2 = np.real(ifft((1j * xi)**2 * q_hat))

        inv1 = 1 / (1 - r5b * (1j * xi)**2)
        dqdx1 = np.real(ifft(inv1 * fft(dqdx)))
        dqdxx = np.real(ifft(inv1 * fft(dq2dx2)))
        q_drhodx = np.real(ifft(inv1 * fft(q * drhodx)))
        q2_dqdx = np.real(ifft(inv1 * fft(dqdx * q**2)))
        q_rho_drhodx = np.real(ifft(inv1 * fft(q * rho * drhodx)))
        dqdx_drhodx = np.real(ifft(inv1 * fft(dqdx * drhodx)))
        q_drhodxx = np.real(ifft(inv1 * fft(q * drho2dx2)))

        inv2 = 1 / (1 - beta11b * (1j * xi)**2)
        drhodx1 = np.real(ifft(inv2 * fft(drhodx)))
        q_dqdx = np.real(ifft(inv2 * fft(q * dqdx)))
        rho_drhodx = np.real(ifft(inv2 * fft(rho * drhodx)))
        drhodxx = np.real(ifft(inv2 * fft(drho2dx2)))
        q_rho_dqdx = np.real(ifft(inv2 * fft(q * rho * dqdx)))
        dqdx2 = np.real(ifft(inv2 * fft(dqdx**2)))
        q_dqdxx = np.real(ifft(inv2 * fft(q * dq2dx2)))
        q2_drhodx = np.real(ifft(inv2 * fft(drhodx * q**2)))
        drhodx2 = np.real(ifft(inv2 * fft(drhodx**2)))
        rho_drhodxx = np.real(ifft(inv2 * fft(rho * drho2dx2)))

        d[0, :] = (r1 * dqdx1 + r2 * dqdxx + r3 * q_drhodx + r4 * q2_dqdx + 
                   r6 * q_rho_drhodx + r7 * dqdx_drhodx + r8 * q_drhodxx) - r6 * p_0 * q_drhodx

        d[1, :] = (beta1 * drhodx1 + beta2 * q_dqdx + beta3 * rho_drhodx  + 
                   beta4 * drhodxx + beta5 * q_rho_dqdx  + beta6 * dqdx2 + 
                   beta7 * q_dqdxx + beta8 * q2_drhodx + beta9 * drhodx2 + beta10 * rho_drhodxx 
                   - beta3 * p_0 * drhodx1- beta5 * p_0 * q_dqdx - beta10 * p_0 * drhodxx)
        return d

    nplt = np.floor((tmax / num_plots) / dt)
    nmax = int(round(tmax / dt))
    
    frames = [v.copy()]
    fframes = [vhat]
    tt = [0]
    progress_bar = tqdm(total=nmax)
    for n in range(1, nmax + 1):
        v_new = rk3(v, xi, rhs, dt)
        v = v_new.copy()
        t = n * dt
        if n % nplt == 0:
            frames.append(v.copy())
            tt.append(t)
            progress_bar.update(nplt)
    progress_bar.close()
    return frames, x, tt, xi

def plot_solution(frames, x, tt, xi):
    fig = plt.figure(figsize=(12, 10))
    axes = fig.add_subplot(211)
    axes2 = fig.add_subplot(212)
    line, = axes.plot(x, frames[0][0, :], lw=3)
    line2, = axes2.plot(x, frames[0][1, :], lw=3)
    xi_max = np.max(np.abs(xi))
    axes2.set_xlabel(r'$x$', fontsize=30)
    axes.set_ylabel(r'$\rho$', fontsize=30)
    axes2.set_ylabel(r'$q$', fontsize=30)
    plt.close()

    def plot_frame(i):
        line.set_data(x, frames[i][0, :])
        line2.set_data(x, frames[i][1, :])
        axes.set_title('t= %.2e' % tt[i])
        axes.set_xlim((x[0], x[-1]))
        axes2.set_xlim((x[0], x[-1]))

    anim = matplotlib.animation.FuncAnimation(fig, plot_frame,
                                              frames=len(frames), interval=100,
                                              repeat=False)
    return HTML(anim.to_jshtml())


In [6]:
cross_section = 'piecewise'
if cross_section == 'pwc_small':
    a = lambda y: np.piecewise(y, [y- np.floor(y) < 0.5, y- np.floor(y) >= 0.5], [1/2, 3/4])
    dady = lambda y: np.zeros_like(y)
    m=6000

elif cross_section == 'variable':
    a = lambda y: (3/5) + (2/5) *np.sin(2.*np.pi*y)
    dady = lambda y: (2/5)*2*np.pi*np.cos(2.*np.pi*y)
    m = 6000

elif cross_section == 'piecewise':
    a = lambda y: np.piecewise(y, [y- np.floor(y) < 0.5, y- np.floor(y) >= 0.5], [1/4, 3/4])
    dady = lambda y: np.zeros_like(y)
    m = 600

In [7]:
delta = 1
C1, C2, C3, C4, C5, C6, C7, C8, C9,C10,C11,C12,C13,C14,C15,C16,avga,ainvavg = C_values(a=a,dady=dady,delta=delta)

In [8]:
print("<a>^2= ",avga**2)
print('C1= ',C1)
print('C2= ',C2)
print('C3= ',C3)
print('C4= ',C4)
print('C5= ',C5)
print('C6= ',C6)
print('C7= ',C7)
print('C8= ',C8)
print('C9= ',C9)
print('C10= ',C10)
print('C11= ',C11)
print('C12= ',C12)
print('C13= ',C13)
print('C14= ',C14)
print('C15= ',C15)
print('C16= ',C16)
avga

<a>^2=  0.25
C1=  -4.3954448375147e-19
C2=  0.006944444444444444
C3=  9.876543209876544
C4=  1.391220826116584e-16
C5=  -52.67489711934156
C6=  -70.23319615912212
C7=  -6.5117701296514086e-18
C8=  0
C9=  -0.003472222222222223
C10=  2.6666666666666665
C11=  -0.01851851851851851
C12=  368.7242798353909
C13=  0.0
C14=  8.88888888888889
C15=  -5.281354331063521e-18
C16=  29.62962962962963


0.5

In [9]:
gamma = 1.4 
kappa =1
p_0 = 0.3
P1 = gamma*kappa*p_0**(gamma-1)
P11= (gamma-1)*gamma*kappa*p_0**(gamma-2)

r1, r2, r3, r4, r5,r5b, r6,r7,r8, beta1, beta2, beta3, beta4, beta5, beta6, beta7,beta8, beta9, beta10, beta11,beta11b = Homogenized_system_coef(C1, C2, C3, C4, C5, C6, C7, C8, C9,C10,C11,C12,C13,C14,C15,C16,avga,ainvavg, p_0, P1, P11, delta)

In [10]:
print("P1 =", P1)
print("P11 =", P11)

P1 = 0.8649211907943767
P11 = 1.153228254392502


In [11]:
print("r1= ",r1)
print("r2= " , r2)
print("r3= " , r3)
print("r4= " , r4)
print("r5= " , r5)
print("r5b= " , r5b)
print("r6= " , r6)
print("r7= " , r7)
print("r8= " , r8)


print("\u03B21= " ,beta1)
print("\u03B22= " ,beta2)
print("\u03B23= " ,beta3)
print("\u03B24= " ,beta4)
print("\u03B25= " ,beta5)
print("\u03B26= " ,beta6)
print("\u03B27= " ,beta7)
print("\u03B28= " ,beta8)
print("\u03B29= " ,beta9)
print("\u03B210= " ,beta10)
print("\u03B211= " ,beta11)
print("\u03B211b= " ,beta11b)

print(1/avga)

r1=  -2.0
r2=  6.593167256272051e-19
r3=  0.0
r4=  -45.6760376089551
r5=  -0.020833333333333336
r5b=  0.04166666666666667
r6=  -14.814814814814817
r7=  -1.8844361608678236e-16
r8=  -1.8844361608678236e-16
β1=  -0.32434544654789127
β2=  -13.333333333333334
β3=  -0.4324605953971883
β4=  -1.0692318889502467e-19
β5=  59.25925925925927
β6=  -2.0239507830705267e-17
β7=  -5.4002216328336363e-17
β8=  34.567901234567884
β9=  -7.365819679435034e-19
β10=  -1.425642518600329e-19
β11=  -0.0033785984015405335
β11b=  0.001095833007253656
2.0


In [ ]:
dt = 9e-05 #1.73/(m/2)**2
print(dt)
frames, x, tt, xi=solve_isentropic_gas_equations(m, dt, num_plots)

9e-05


  4%|█████▉                                                                                                                                                | 22222.0/555556 [01:05<26:09, 339.77it/s]

In [ ]:
plot_solution(frames, x, tt, xi)